<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/plot_before_after_storm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt

#%matplotlib inline

In [ ]:
def get_file_name(dt, year, myBasin, hurr_name, fSuff_base = 'GLORYS12.nc'):

  input_data_path = '/content/drive/MyDrive/datasets/hurr/{}/'.format(year)
  fPref = input_data_path + '{}_{}_{}_'.format(year, myBasin, hurr_name.upper())

  if dt == 0:
    fSuff = ""+ fSuff_base
  elif dt > 0:
    fSuff = str(abs(dt)).zfill(3) + "days_after_" + fSuff_base
  elif dt < 0:
    fSuff =  str(abs(dt)).zfill(3) + "days_before_" + fSuff_base

  return (fPref+fSuff)

In [ ]:
# Mount drive - to rw files
from google.colab import drive
drive.mount('/content/drive')

## User Inputs

In [ ]:
myBasin = 'north_atlantic'
year = 2024
hurr_name='milton'

### Read downloaded track data

In [ ]:
input_data_path = '/content/drive/MyDrive/datasets/hurr/{}/'.format(year)
print(input_data_path)

track_file  = input_data_path + '{}_{}_{}.nc'.format(year, myBasin, hurr_name.upper())
ds_track = xr.open_dataset(track_file)

### Plot track and other hurricane info

In [ ]:
width, height, DPI = [8, 6, 80]
dLon, dLat= [2, 2]
#
xMin, xMax, yMin, yMax = [ds_track.lon.min()-dLon, ds_track.lon.max()+dLon, ds_track.lat.min()-dLat, ds_track.lat.max()+dLat]

# Date of minimum MSLP/highest intensity
tMin=ds_track.mslp.where(ds_track.mslp==ds_track.mslp.min(), drop=True).time[0]
print("Min MSLP {} on: {}".format(ds_track.mslp.min().values, tMin.values.astype('str').split('T')[0]))
latMin, lonMin = [ds_track.sel(time=tMin).lat.values, ds_track.sel(time=tMin).lon.values]
print("Min SLP at: [lat, lon]=\t{}\t{}".format(latMin, lonMin))
#

fig = plt.figure(figsize=(width,height), dpi=DPI)

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.LAND, zorder=1, edgecolor='black', facecolor='0.9')
ax.set_extent([xMin, xMax, yMin, yMax], crs=ccrs.PlateCarree())

im=ds_track.plot.scatter(x='lon', y='lat', ax=ax, transform=ccrs.PlateCarree(), s=ds_track.vmax, c=ds_track.mslp,
                         marker='o', edgecolors='none', cmap='plt.cm.gist_ncar')

ax.text(ds_track.lon[0].values, ds_track.lat[0].values, "{}".format(ds_track.time[0].values.astype(str).split('T')[0]),
        transform=ccrs.PlateCarree(), fontsize=8)
ax.text(ds_track.lon[-1].values-dLon, ds_track.lat[-1].values, "{}".format(ds_track.time[-1].values.astype(str).split('T')[0]),
        transform=ccrs.PlateCarree(), fontsize=8)

ax.text(lonMin, latMin, "{}".format(tMin.values.astype('str').split('T')[0]), transform=ccrs.PlateCarree(), fontsize=16)
ax.scatter(lonMin, latMin, s=20, marker='x', c='r', transform=ccrs.PlateCarree())

plt.colorbar(im, pad=0.01, shrink=0.6, label='MSLP (hPa)')
ax.text(xMin+1, yMax-1, str(year)+" "+hurr_name.upper(), transform=ccrs.PlateCarree(), fontsize=12)

### Plot before/after storm
- AVISO (ADT)
- GLORYS12 (SSH)
- GLORYS12 (thetao top level)
- GLORYS12 (so top level)


In [ ]:
#!ls /content/drive/MyDrive/datasets/hurr/2024/*MILTON*AVISO*

fPref = input_data_path + '{}_{}_{}_'.format(year, myBasin, hurr_name.upper())
fSuff_base, vName = ['AVISO.nc', 'adt']
#print(fPref)

dts = [-5, -1, 0, 1, 5]
for dt in dts:
  #print(dt)
  if dt == 0:
    fSuff = ""+ fSuff_base
  elif dt > 0:
    fSuff = str(abs(dt)).zfill(3) + "days_after_" + fSuff_base
  elif dt < 0:
    fSuff =  str(abs(dt)).zfill(3) + "days_before_" + fSuff_base

  fName=fPref+fSuff
  #print(fName)
  ds_euler_aviso=xr.open_dataset(fName)
  if dt == 0:
    ds_euler_aviso[vName].plot(x="time", label="dt="+str(dt), lw=4)
  else:
    ds_euler_aviso[vName].plot(x="time", label="dt="+str(dt))

plt.title("")
plt.legend(ncols=1)

In [ ]:
#!ls /content/drive/MyDrive/datasets/hurr/2024/*MILTON*GLO*

fPref = input_data_path + '{}_{}_{}_'.format(year, myBasin, hurr_name.upper())
fSuff_base = 'GLORYS12.nc'
vNames = ['zos', 'thetao', 'so']
#print(fPref)

fig, ax_all = plt.subplots(nrows=len(vNames), ncols=1, sharex=True, figsize=(8, 12), clear=True)

dts = [-5, -1, 0, 1, 5]
for dt in dts:
  #print(dt)
  if dt == 0:
    fSuff = ""+ fSuff_base
  elif dt > 0:
    fSuff = str(abs(dt)).zfill(3) + "days_after_" + fSuff_base
  elif dt < 0:
    fSuff =  str(abs(dt)).zfill(3) + "days_before_" + fSuff_base

  fName=fPref+fSuff
  #print(fName)
  ds_euler_glo12=xr.open_dataset(fName)

  for ax, vName in zip(ax_all, vNames):

    ax.axvline(x=ds_track.mslp.where(ds_track.mslp==ds_track.mslp.min(), drop=True).time,
               ls='--', color='k', alpha= 0.5)

    if vName == vNames[0]:
      ax2 = ax.twinx()
      ax2.plot_date(ds_track.time, ds_track.mslp, ls='-', c='k', alpha=0.5)
      ax2.set_ylabel('MSLP (hPa)')

    if dt == 0:
      ds_euler_glo12[vName].isel(depth=0).plot(x="time", label=str(dt)+" days", lw=4, ax=ax)
    else:
      ds_euler_glo12[vName].isel(depth=0).plot(x="time", label=str(dt)+" days", lw=2, ax=ax)
    ax.set_title("")
    ax.legend(ncols=1)

## Plot (subsurface) temperature and salinity before/after storm:

- [dt= -5] 5 days before.
- 1 day before.
- Along storm (dt=0).
- 1 day after.
- 5 days after.

In [ ]:
vNames = ['thetao', 'so']
temp_cmin, temp_cmax, temp_cmap=[12, 32, 'RdBu_r']
salt_cmin, salt_cmax, salt_cmap=[35, 38, 'Spectral_r']

for dts in [-5, -1, 0, 1, 5]:
  fName=get_file_name(dts, year, myBasin, hurr_name)
  print(fName)
  ds_euler_glo12=xr.open_dataset(fName)

  fig, ax_all = plt.subplots(nrows=1, ncols=len(vNames), sharex=True, figsize=(12, 4), clear=True)
  for ax, vName in zip(ax_all, vNames):
    if vName == "thetao":
      im=ds_euler_glo12[vName].sel(depth=slice(0, 500)).plot.contourf(ax=ax, x="time", y="depth", yincrease=False,
                                                           vmin=temp_cmin, vmax=temp_cmax, cmap=temp_cmap)
    else:
      im=ds_euler_glo12[vName].sel(depth=slice(0, 500)).plot.contourf(ax=ax, x="time", y="depth", yincrease=False,
                                                           vmin=salt_cmin, vmax=salt_cmax, cmap=salt_cmap)
      ax.set_ylabel("")

  ax.set_title("")